In [1]:
!which python
!nvidia-smi -L

/apps/anaconda3/bin/python


GPU 0: Tesla V100-PCIE-32GB (UUID: GPU-73e475a7-966f-91ae-6652-4163028a8284)


In [2]:
# !python /home/FYP/siddhant005/fyp/src/inference.py -d ml-100k --pretrain '/home/FYP/siddhant005/fyp/log/saved_models/ml-100k/190_checkpoint.128_30_200_2_0.3_disentangle_2_30_32_32_0.001.pth.tar'

In [3]:
from src.inference import InferenceEngine
from src.utils.data_utils import Data
from src.utils.main_utils import parse_opt

args = parse_opt()[0]
print(args.data)
data = Data(args.data, args, split=False)
inference = InferenceEngine(args, data)

user_id = 259
print(f'Running inference for user {user_id}...')
uit = inference.run(user_id)
print(uit)

ml-100k
Running inference for user 259...
The mapped user id 101 is for 259 real user id
(array([101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101,
       101, 101, 101, 101, 101, 101, 101]), array([ 177,  289,  153,  361,  321,  732, 1006,   60,  971,  140,  113,
        136, 1054,  311,  347,  240,  111,   52,   31,  297,  320,   88,
        367,  491,  492,  571,  355, 1008,   68,  403,   95,  719,  203,
        175,  427,  161,  365,  181,   25,  115,  452,  310,  241,  285,
        462,  269]), array([8.74724710e+08, 8.74724727e+08, 8.74724754e+08, 8.74724781e+08,
       8.74724843e+08, 8.74724882e+08, 8.74724882e+08, 8.74724905e+08,
       8.74724937e+08, 8.74724988e+08, 8.74725081e+08, 8.74725120e+08,
       8.74725264e+08, 8.74725386e+08, 8.74725485e+08, 8.74725485e+08,
       8.74809057e+08, 8.74809057e+08, 8.74809091e+08

In [6]:
src, dst, ts = uit[0], uit[1], uit[2]

In [7]:
import networkx as nx
def make_nx_graph( src, dst, ts):
    B = nx.Graph()
    B.add_nodes_from(src, bipartite=1)
    B.add_nodes_from(dst, bipartite=0)
    
    edges = [(src[i], dst[i]) for i in range(len(dst))]
    
    B.add_edges_from(edges)

    top = nx.bipartite.sets(B)[0]
    pos = nx.bipartite_layout(B, top, align='horizontal')

    return B, pos

In [10]:
import plotly.graph_objects as go

G, pos = make_nx_graph(src, dst, ts)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))


node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

KeyError: 'pos'